In [1]:
#Imports and GCS Setup: This cell imports the libraries and sets up the path to your data in the GCS bucket.

import os
import pandas as pd
import numpy as np
import pickle
from deepface import DeepFace
from google.cloud import storage
import gcsfs
from tqdm import tqdm # A library to show progress bars

# --- CONFIGURATION ---
# Make sure to replace this with your actual bucket name
BUCKET_NAME = "computer-vision-security-sys-data-hj" 
# --- END CONFIGURATION ---

# Path to the raw faces data in your bucket
GCS_FACES_PATH = f"gs://{BUCKET_NAME}/raw/faces/"

# This allows pandas to read directly from GCS
fs = gcsfs.GCSFileSystem()

print("Setup complete. GCS path is set to:", GCS_FACES_PATH)

Setup complete. GCS path is set to: gs://computer-vision-security-sys-data-hj/raw/faces/


In [2]:
# Discover Employee Data: This cell will list all the employee sub-folders you created in your GCS bucket.

# List all the directories (one for each employee) in the GCS path
employee_names = [os.path.basename(f) for f in fs.ls(GCS_FACES_PATH) if fs.isdir(f)]

if not employee_names:
    print("❌ Error: No employee folders found in your GCS bucket.")
    print("Please ensure your face data has been uploaded correctly.")
else:
    print("✅ Found employee folders:", employee_names)

✅ Found employee folders: ['background', 'himanshu_jangra', 'rohit_agarwal', 'shivani_shahi']


In [ ]:
# Generate, Average, and Store Embeddings
import tempfile

# The model we'll use for generating embeddings
# VGG-Face and Facenet are both excellent choices.
# MODEL_NAME = "DeepFace"
MODEL_NAME = "ArcFace"
# MODEL_NAME = "VGG-Face"
# MODEL_NAME = "FaceNet512"

employee_embeddings = {}

# Use tqdm for a nice progress bar
for name in tqdm(employee_names, desc="Processing Employees"):
    employee_folder_path = os.path.join(GCS_FACES_PATH, name)
    image_files = fs.ls(employee_folder_path)

    # total_images = image_files.count()
    current_employee_embeddings = []
    not_processed = 0
    
    for image_path in tqdm(image_files, desc=f"  -> {name}", leave=False):
        # The full GCS path for the image
        full_image_path = f"gs://{image_path}"

        # 1. Create a temporary file but tell it not to delete on close
        tmp_file = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False)
        local_path = tmp_file.name
        
        # 2. Immediately close the file to release the Windows lock
        tmp_file.close()

        try:
            # Download the GCS file to the temporary local path
            fs.get(full_image_path, local_path)
            
            # DeepFace's represent function does the magic:
            # 1. Detects the face in the image.
            # 2. Generates the vector embedding.
            embedding_obj = DeepFace.represent(
                img_path=local_path,
                model_name=MODEL_NAME,
                enforce_detection=True # Ensures we don't process images without a face
            )
            
            # We only need the embedding vector itself
            current_employee_embeddings.append(embedding_obj[0]["embedding"])

        except ValueError as e:
            # This error is often thrown if a face isn't detected in an image
            print(f"\nWarning: Could not process {os.path.basename(image_path)}. Reason: {e}")
            not_processed = not_processed + 1
        except Exception as e:
            print(f"\nAn unexpected error occurred with {os.path.basename(image_path)}: {e}")

    if current_employee_embeddings:
        # --- NEW LOGIC: Create a gallery of up to 10 embeddings ---
        gallery_size = 10
        if len(current_employee_embeddings) > gallery_size:
            # If we have more than 10, select a random sample
            import random
            selected_embeddings = random.sample(current_employee_embeddings, gallery_size)
        else:
            # If we have 10 or fewer, just use all of them
            selected_embeddings = current_employee_embeddings
    
        employee_embeddings[name] = selected_embeddings
        print(f"\n✅ Created an embedding gallery with {len(selected_embeddings)} entries for {name}")
        print(f"Images not processed: {not_processed}")
    else:
        print(f"\n❌ Failed to generate any embeddings for {name}. Please check the images.")

print("\n--- All employees processed. ---")

  -> background:   3%|█████▋                                                                                                                                                                  | 17/500 [00:13<03:10,  2.54it/s]


  -> background:   5%|████████                                                                                                                                                                | 24/500 [00:16<03:07,  2.54it/s]


  -> background:  10%|████████████████▏                                                                                                                                                       | 48/500 [00:26<02:33,  2.94it/s]


  -> background:  14%|████████████████████████▏                                                                                                                                               | 72/500 [00:35<02:19,  3.08it/s]


  -> background:  33%|██████████████████████████████████████████████████████▊                                                                                                                | 164/500 [01:13<02:12,  2.53it/s]

...


  -> rohit_agarwal:  52%|████████████████████████████████████████████████████████████████████████████████████▌                                                                               | 258/500 [02:37<02:12,  1.82it/s]


  -> rohit_agarwal:  53%|██████████████████████████████████████████████████████████████████████████████████████▎                                                                             | 263/500 [02:40<02:12,  1.79it/s]


  -> rohit_agarwal:  55%|█████████████████████████████████████████████████████████████████████████████████████████▌                                                                          | 273/500 [02:45<02:03,  1.84it/s]


  -> rohit_agarwal:  55%|██████████████████████████████████████████████████████████████████████████████████████████▊                                                                         | 277/500 [02:48<01:58,  1.89it/s]


  -> rohit_agarwal:  57%|█████████████████████████████████████████████████████████████████████████████████████████████▍                                                                      | 285/500 [02:53<02:18,  1.55it/s]

In [ ]:
# Save the Face Database

# The location where your main processor script will look for the embeddings
SAVE_PATH = "../src/processor/face_embeddings.pkl"

with open(SAVE_PATH, "wb") as f:
    pickle.dump(employee_embeddings, f)

print(f"✅ Face database saved successfully to: {SAVE_PATH}")